In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torchvision import datasets, models
from torch.utils.data.sampler import SubsetRandomSampler
import os
import random
import numpy as np
import time
import matplotlib.pyplot as plt

In [2]:

def process_data(datadir, valid_size = 0.25, batch_size = 128):
    
    train_transforms = transforms.Compose([transforms.Resize(227),transforms.ToTensor(),])
    test_transforms = transforms.Compose([transforms.Resize(227),transforms.ToTensor(),])
    
    train_data = datasets.ImageFolder(datadir, transform=train_transforms)
    test_data = datasets.ImageFolder(datadir, transform=test_transforms)
    
    num_train = len(train_data)
    indices = list(range(num_train))
    split = int(np.floor(valid_size*num_train))
    np.random.shuffle(indices)
    train_idx,test_idx = indices[split:], indices[:split]
    train_sampler = SubsetRandomSampler(train_idx)
    test_sampler = SubsetRandomSampler(test_idx)
    
    trainloader = torch.utils.data.DataLoader(train_data, sampler = train_sampler, batch_size = batch_size)
    testloader = torch.utils.data.DataLoader(test_data, sampler = test_sampler, batch_size = batch_size)
    
    return trainloader,testloader

In [3]:
data_dir = "./data/mendeley"

train_loader,test_loader = process_data(data_dir)

In [4]:
class CNN_Model(nn.Module):
    def __init__(self):
        super().__init__()  
    
        self.features = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=7, padding=1, stride=2),
            nn.LocalResponseNorm(size=5, alpha=0.0001, beta=0.75, k=2),
            nn.ReLU(inplace=False),
            nn.Conv2d(32, 64, kernel_size=3, padding=1, stride=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=False),
            nn.MaxPool2d(kernel_size=2, stride=2),          
            nn.Conv2d(64, 96, kernel_size=3, padding=1, stride=1),
            nn.BatchNorm2d(96),
            nn.ReLU(inplace=False),
            nn.MaxPool2d(kernel_size=2, stride=2),           
            nn.Conv2d(96, 128, kernel_size=5, padding=2, stride=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=False),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(128, 128, kernel_size=3, padding=1, stride=1),
            nn.ReLU(inplace=False),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )
        
        self.classifier = nn.Sequential(
            nn.Linear(6272, 2048),
            nn.ReLU(inplace=False),
            nn.Dropout(p=0.5, inplace=True),
            nn.Linear(2048, 512),
            nn.ReLU(inplace=False),
            nn.Dropout(p=0.5, inplace=True),
            nn.Linear(512, 2),
        )    
        self.init_bias()
    
    def init_bias(self):
        for layer in self.features:
            if isinstance(layer, nn.Conv2d):
                nn.init.normal_(layer.weight, mean=0, std=0.01)
                nn.init.constant_(layer.bias, 0)        
    
    def forward(self, x):
        x = self.features(x)
        x = x.view(x.shape[0], -1)
        x = self.classifier(x)
        return x


In [5]:
torch.cuda.empty_cache()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = CNN_Model().to(device)

In [6]:
optimizer = optim.Adam(model.parameters())
criterion = nn.CrossEntropyLoss()

In [7]:
def accuracy(fp,y):
    preds = fp.max(1, keepdim=True)[1]
    correct = preds.eq(y.view_as(preds)).sum()
    acc = correct.float()/preds.shape[0]
    return acc

In [8]:
def train(model, device, data_itr, optimizer, criterion):
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    for x,y in data_itr:
        x = x.to(device)
        y = y.to(device)
        
        optimizer.zero_grad()
        fp = model(x)
        loss = criterion(fp,y)
        acc = accuracy(fp,y)
        
        loss.backward()
        optimizer.step()
        
        epoch_loss += float(loss.item())
        epoch_acc += float(acc.item())
    return epoch_loss / len(data_itr), epoch_acc / len(data_itr)

In [9]:
def evaluate(model, device, data_itr, criterion):
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    with torch.no_grad():
        for x,y in data_itr:
            x = x.to(device)
            y = y.to(device)
            
            fp = model(x)
            loss = criterion(fp,y)
            acc = accuracy(fp,y)
            
            epoch_loss += loss.item()
            epoch_acc += acc.item()
    
    return epoch_loss / len(data_itr), epoch_acc / len(data_itr)

In [ ]:
epochs = 1500
SAVE_DIR = 'models'
CKPT_SAVE_PATH = os.path.join(SAVE_DIR, 'cnn_crack_mendeleys.pth.tar')
MODEL_SAVE_PATH = os.path.join(SAVE_DIR, 'cnn_crack_mendeleys_model.pt')

best_val_loss = float('inf')

# if not os.path.isdir(f'{SAVE_DIR}'):
#     os.makedirs(f'{SAVE_DIR}')

Epochs = []
Training_loss = []
Training_accuracy = []
Validation_loss = []
Validation_accuracy = []
    
for epoch in range(epochs):
    to = time.time()
    train_loss,train_acc = train(model, device, train_loader, optimizer, criterion)
    val_loss,val_acc = evaluate(model, device, test_loader, criterion)
    tepoch = time.time() - to
    
    Epochs.append(epoch+1)
    Training_loss.append(train_loss)
    Training_accuracy.append(train_acc)
    Validation_accuracy.append(val_acc)
    Validation_loss.append(val_loss)
    
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        best_epoch = epoch+1
        torch.save(model.state_dict(), MODEL_SAVE_PATH)
        
    if epoch%50 == 0:
        torch.save({
            'epoch': Epochs,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'train_loss': Training_loss,
            'valid_loss': Validation_loss,
            'train_acc': Training_accuracy,
            'val_acc': Validation_accuracy,
            }, CKPT_SAVE_PATH)
        
    print("Epoch: %4d | Training loss: %.3f | Training Accuracy: %5.2f | Valid_loss: %.3f | Valid_accuracy: %5.2f | Time: %.3fsec" %(epoch+1, train_loss, train_acc*100, val_loss, val_acc*100, tepoch))

print((best_val_loss,best_epoch))

Epoch:    1 | Training loss: 0.070 | Training Accuracy: 97.80 | Valid_loss: 0.042 | Valid_accuracy: 98.52 | Time: 143.263sec
Epoch:    2 | Training loss: 0.022 | Training Accuracy: 99.37 | Valid_loss: 0.030 | Valid_accuracy: 99.44 | Time: 141.905sec
Epoch:    3 | Training loss: 0.020 | Training Accuracy: 99.50 | Valid_loss: 0.017 | Valid_accuracy: 99.51 | Time: 142.029sec
Epoch:    4 | Training loss: 0.012 | Training Accuracy: 99.67 | Valid_loss: 0.314 | Valid_accuracy: 86.53 | Time: 141.998sec
Epoch:    5 | Training loss: 0.010 | Training Accuracy: 99.74 | Valid_loss: 0.017 | Valid_accuracy: 99.59 | Time: 141.943sec
Epoch:    6 | Training loss: 0.010 | Training Accuracy: 99.73 | Valid_loss: 0.007 | Valid_accuracy: 99.82 | Time: 141.857sec
Epoch:    7 | Training loss: 0.007 | Training Accuracy: 99.78 | Valid_loss: 0.010 | Valid_accuracy: 99.74 | Time: 142.126sec
Epoch:    8 | Training loss: 0.007 | Training Accuracy: 99.80 | Valid_loss: 0.010 | Valid_accuracy: 99.79 | Time: 142.099sec


Epoch:   67 | Training loss: 0.004 | Training Accuracy: 99.92 | Valid_loss: 0.020 | Valid_accuracy: 99.84 | Time: 141.071sec
Epoch:   68 | Training loss: 0.002 | Training Accuracy: 99.94 | Valid_loss: 0.020 | Valid_accuracy: 99.77 | Time: 140.867sec
Epoch:   69 | Training loss: 0.000 | Training Accuracy: 99.99 | Valid_loss: 0.029 | Valid_accuracy: 99.83 | Time: 140.470sec
Epoch:   70 | Training loss: 0.000 | Training Accuracy: 99.99 | Valid_loss: 0.026 | Valid_accuracy: 99.84 | Time: 140.422sec
Epoch:   71 | Training loss: 0.000 | Training Accuracy: 99.99 | Valid_loss: 0.028 | Valid_accuracy: 99.84 | Time: 140.500sec
Epoch:   72 | Training loss: 0.000 | Training Accuracy: 99.99 | Valid_loss: 0.026 | Valid_accuracy: 99.77 | Time: 140.711sec
Epoch:   73 | Training loss: 0.001 | Training Accuracy: 99.97 | Valid_loss: 0.023 | Valid_accuracy: 99.79 | Time: 141.025sec
Epoch:   74 | Training loss: 0.000 | Training Accuracy: 100.00 | Valid_loss: 0.028 | Valid_accuracy: 99.85 | Time: 140.938sec

Epoch:  133 | Training loss: 0.000 | Training Accuracy: 100.00 | Valid_loss: 0.040 | Valid_accuracy: 99.82 | Time: 139.845sec
Epoch:  134 | Training loss: 0.000 | Training Accuracy: 100.00 | Valid_loss: 0.046 | Valid_accuracy: 99.81 | Time: 140.021sec
Epoch:  135 | Training loss: 0.000 | Training Accuracy: 100.00 | Valid_loss: 0.039 | Valid_accuracy: 99.82 | Time: 139.980sec
Epoch:  136 | Training loss: 0.000 | Training Accuracy: 100.00 | Valid_loss: 0.041 | Valid_accuracy: 99.80 | Time: 139.948sec
Epoch:  137 | Training loss: 0.000 | Training Accuracy: 100.00 | Valid_loss: 0.044 | Valid_accuracy: 99.80 | Time: 139.792sec
Epoch:  138 | Training loss: 0.015 | Training Accuracy: 99.85 | Valid_loss: 0.026 | Valid_accuracy: 99.76 | Time: 139.852sec
Epoch:  139 | Training loss: 0.002 | Training Accuracy: 99.96 | Valid_loss: 0.024 | Valid_accuracy: 99.78 | Time: 139.570sec
Epoch:  140 | Training loss: 0.000 | Training Accuracy: 100.00 | Valid_loss: 0.027 | Valid_accuracy: 99.78 | Time: 139.5

In [ ]:
#Plotting graphs
#Loss curves
plt.plot(Epochs, Training_loss, 'g', label='Training Loss')
plt.plot(Epochs, Validation_loss, 'b', label='Validation Loss')
plt.title('Training and Validation Losses')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.savefig('loss.png')
plt.show()

In [ ]:
#Accuracy Curves 
plt.plot(Epochs, Training_accuracy, 'g', label='Training Accuracy')
plt.plot(Epochs, Validation_accuracy, 'b', label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.savefig('accuracy.png')
plt.show()